In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate();

using LinearAlgebra
using BlockDiagonals
using ForwardDiff
using Plots
using Random;
using Printf

using RobotZoo: Quadrotor
using RobotDynamics
using MeshCat
using ColorTypes
using TrajOptPlots
using GeometryBasics: Point, Cylinder
using CoordinateTransformations
using Rotations

include("../quaternion-stuff.jl")

  Activating environment at `~/Git/solvers/tinympc/tinympc-dev-julia/quat_mpc_state/Project.toml`


q_distance (generic function with 1 method)

In [117]:
# Quadrotor parameters
m = 0.035
ℓ = 0.046
J = [16.6e-6 0.83e-6 0.72e-6
    0.83e-6 16.6e-6 1.8e-6
    0.72e-6 1.8e-6 29.3e-6];
g = 9.81
# thrustToTorque = 0.005964552
thrustToTorque = 0.0008
ℓ = 0.046 / sqrt(2)
scale = 65535
kt = 2.245365e-6 * scale # u is PWM in range [0...1]
km = kt * thrustToTorque #4.4733e-8

freq1 = 50
h = 1 / freq1 #50 Hz

nx = 12     # number of states (linearized)
nu = 4     # number of controls

function quad_dynamics(x, u)
    r = x[1:3]
    q = x[4:7] / norm(x[4:7])
    v = x[8:10]
    ω = x[11:13]
    Q = qtoQ(q)
  
    ṙ = v
    q̇ = 0.5 * L(q) * H * ω
  
    v̇ = [0; 0; -g] + (1 / m) * Q * [zeros(2, 4); kt * ones(1, 4)] * u
  
    ω̇ = J \ (-hat(ω) * J * ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km] * u)
  
    return [ṙ; q̇; v̇; ω̇]
  end
  
  function quad_dynamics_rk4(x, u, h)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5 * h * f1, u)
    f3 = quad_dynamics(x + 0.5 * h * f2, u)
    f4 = quad_dynamics(x + h * f3, u)
    xn = x + (h / 6.0) * (f1 + 2 * f2 + 2 * f3 + f4)
    xn[4:7] .= xn[4:7] / norm(xn[4:7]) #re-normalize quaternion
    return xn
  end

quad_dynamics_rk4 (generic function with 1 method)

In [3]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/sam/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


false

In [4]:
# Create reference trajectory

Tfinal = 5.0
NTOTAL = Int(Tfinal/h)+1

# Reference trajectory
X̃ref = [zeros(nx) for i = 1:NTOTAL]
scale = 5
# Xref = [[sin(2*t); 1.5*cos(t); 0.7*sin(3*t); 1; zeros(9)] for t in range(-pi/2*2, 3*pi/2*2, length=NTOTAL)] # mega pringle
# Xref = [[sin(scale*t)*cos(t); cos(scale*t)*cos(t); 0.75*sin(t); 1; zeros(9)] for t in range(0, 2*pi, length=NTOTAL)] # ball
# Xref = [[0.05; t; 0.5;   1; zeros(3);   zeros(6)] for t = range(0, 5, length = NTOTAL)]; # straight line
# Xref = [zeros(nx+1) for i = 1:NTOTAL] # origin




# Xref = [[0.0; t; 0.5;   1; zeros(3);   zeros(6)] for t = range(0, 5, length = NTOTAL)]; # straight line with barrel roll
# num_ref_points = length(Xref)

# # start barrel roll at 2/5 and end at 3/5 of trajectory
# start_barrel_roll_ratio = 2.0/5.0
# end_barrel_roll_ratio = 3.0/5.0
# num_point_in_barrel_roll = Int(floor(num_ref_points * (end_barrel_roll_ratio - start_barrel_roll_ratio)))
# barrel_roll_angles = LinRange(0.0, 2*pi, num_point_in_barrel_roll) # create rough desired angles for barrel roll
# quats = [[cos(α/2), 0, sin(α/2), 0] for α in barrel_roll_angles]
# start_barrel_roll_index = Int(floor(num_ref_points*start_barrel_roll_ratio))
# for i in 1:num_point_in_barrel_roll
#     Xref[i+start_barrel_roll_index][4:7] = quats[i]
# end

# # zigzag attitude trajectory
# zigzag_1_ratio = 1.0/6.0
# zigzag_2_ratio = 2.0/6.0
# zigzag_3_ratio = 4.0/6.0
# zigzag_4_ratio = 5.0/6.0
# num_points_per_zig = Int(floor(num_ref_points * (zigzag_4_ratio - zigzag_1_ratio)/4))
# zigzag_1_angles = LinRange(0.0, pi/4, num_points_per_zig) # create rough desired angles for barrel roll
# zigzag_2_angles = LinRange(pi/4, -pi/4, num_points_per_zig*2) # create rough desired angles for barrel roll
# zigzag_3_angles = LinRange(-pi/4, 0, num_points_per_zig) # create rough desired angles for barrel roll
# quats_zigzag_1 = [[cos(α/2), 0, sin(α/2), 0] for α in zigzag_1_angles]
# quats_zigzag_2 = [[cos(α/2), 0, sin(α/2), 0] for α in zigzag_2_angles]
# quats_zigzag_3 = [[cos(α/2), 0, sin(α/2), 0] for α in zigzag_3_angles]
# quats_zigzag = [quats_zigzag_1; quats_zigzag_2; quats_zigzag_3]
# start_zigzag_index = Int(floor(num_ref_points*zigzag_1_ratio))
# for i in 1:num_points_per_zig*4
#     Xref[i+start_zigzag_index][4:7] = quats_zigzag[i]
# end



# power loop
Xref_lead_in = [[t; 0.0; 0.5;   1; zeros(3);   zeros(6)] for t = range(0, 2.5, length = Int(floor(NTOTAL/4))+1)];
num_points_for_loop = Int(floor(NTOTAL*1/2))
Xref_loop = [[2.5 + cos(theta - pi/2); 0; 1.5 + sin(theta - pi/2);   
              cos(-theta/2); 0; sin(-theta/2); 0;
              zeros(6)] for theta = range(0, 2*pi, length = num_points_for_loop)]
# Xref_loop = [[2.5 + cos(theta - pi/2); 0; 1.5 + sin(theta - pi/2);   
#             1; 0; 0; 0;
#             zeros(6)] for theta = range(0, 2*pi, length = num_points_for_loop)]
Xref_lead_out = [[t; 0.0; 0.5;   1; zeros(3);   zeros(6)] for t = range(2.5, 5, length = Int(floor(NTOTAL/4))+1)];
Xref = [Xref_lead_in; Xref_loop; Xref_lead_out]
num_ref_points = length(Xref)


Uref = [zeros(nu) for i = 1:NTOTAL-1]

# Compute reference velocity from reference position
for i = 1:NTOTAL-1
    Xref[i][8:10] = (Xref[i+1][1:3] - Xref[i][1:3])/h
end

# Convert (13 state) Xref to reduced form (12 state) X̃ref
for k = 1:NTOTAL
    x = Xref[k]
    X̃ref[k] = [x[1:3]; qtorp(x[4:7]); x[8:10]; x[11:13]]
end

X̃ref = hcat(X̃ref...)
Uref = hcat(Uref...)

# Visualize reference
delete!(vis["xHistLine"])
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.75, 0.1, 0.1)), "Line"))
        # LineBasicMaterial(color=Colors.RGBA(0.0, 0.45, 1.0)), "Line"))

MeshCat Visualizer with path /meshcat/XrefLine at http://127.0.0.1:8700

In [5]:
# Visualize trajectory attitude

x_arrow_material = LineBasicMaterial(color=Colors.RGBA(0.55,0.1,0.1,1.0))
x_shaft = Cylinder(zero(Point{3, Float64}), Point(0.2, 0.0, 0.0), 0.01)
setobject!(vis["coordinate_frame_ref"][:xarrow][:shaft], x_shaft, x_arrow_material)
x_head = Cone(Point(0.2, 0.0, 0.0), Point(0.25, 0.0, 0.0), 0.025)
setobject!(vis["coordinate_frame_ref"][:xarrow][:head], x_head, x_arrow_material)

y_arrow_material = LineBasicMaterial(color=Colors.RGBA(0.1,0.55,0.1,1.0))
y_shaft = Cylinder(zero(Point{3, Float64}), Point(0.0, 0.2, 0.0), 0.01)
setobject!(vis["coordinate_frame_ref"][:yarrow][:shaft], y_shaft, y_arrow_material)
y_head = Cone(Point(0.0, 0.2, 0.0), Point(0.0, 0.25, 0.0), 0.025)
setobject!(vis["coordinate_frame_ref"][:yarrow][:head], y_head, y_arrow_material)

z_arrow_material = LineBasicMaterial(color=Colors.RGBA(0.1,0.1,0.55,1.0))
z_shaft = Cylinder(zero(Point{3, Float64}), Point(0.0, 0.0, 0.2), 0.01)
setobject!(vis["coordinate_frame_ref"][:zarrow][:shaft], z_shaft, z_arrow_material)
z_head = Cone(Point(0.0, 0.0, 0.2), Point(0.0, 0.0, 0.25), 0.025)
setobject!(vis["coordinate_frame_ref"][:zarrow][:head], z_head, z_arrow_material)

# delete!(vis[coordinate_frame_name])

for i in 1:length(Xref)
    settransform!(vis["coordinate_frame_ref"], Translation(Xref[i][1:3]) ∘ LinearMap(UnitQuaternion(rptoq(X̃ref[4:6,i]))))
    sleep(0.01)
end

In [6]:
include("../LibTinyMPC.jl")

# Goal state (initialize solver with cache corresponding to
# this linearization. Can add more linearization caches later)
quat_lin_1 = [1.0; 0; 0; 0]
x_lin_1 = [zeros(3); quat_lin_1; zeros(3); zeros(3)];
u_lin_1 = (m * g / kt / 4) * ones(4)  # m = 30g and max thrust = 60g

# Linearize dynamics about hover
A = ForwardDiff.jacobian(x -> quad_dynamics_rk4(x, u_lin_1, h), x_lin_1)
B = ForwardDiff.jacobian(u -> quad_dynamics_rk4(x_lin_1, u, h), u_lin_1);
Ã = Array(E(quat_lin_1)' * A * E(quat_lin_1))
B̃ = Array(E(quat_lin_1)' * B);

max_dev_x = [0.01; 0.01; 0.01; 0.001; 0.001; 0.001; 0.5; 0.5; 0.5; 0.7; 0.7; 0.2] / 1
max_dev_u = [0.5; 0.5; 0.5; 0.5] / 10
Q = diagm(1 ./ (max_dev_x .^ 2))
R = diagm(1 ./ (max_dev_u .^ 2))

Nh = 20

solver = TinySolver(x_lin_1, Ã, B̃, Q, R, Nh, X̃ref, Uref);


# Create more caches for different attitudes
linearization_angles = LinRange(0.0, 2*pi, 17)
quats_linearize = [[cos(α/2), 0, sin(α/2), 0] for α in linearization_angles[1:end-1]]
new_caches = []
for quat in quats_linearize

    # Compute stuff
    x_lin = [zeros(3); quat; zeros(3); zeros(3)]
    u_lin = (m * g / kt / 4) * ones(4)  # m = 30g and max thrust = 60g


    # Visualize stuff
    settransform!(vis["coordinate_frame_ref"], Translation(0.0, 0.0, 0.5) ∘ LinearMap(UnitQuaternion(quat)))

    xbar = RBState(quad_model, x_lin[:,1])
    if quad_model.ned
        rq = position(xbar)
        vq = linear_velocity(xbar)
        rq = SA[rq[1],-rq[2],-rq[3]]
        vq = SA[vq[1],-vq[2],-vq[3]]
        xbar = RBState(rq, RotX(pi)*orientation(xbar), vq, angular_velocity(xbar)) 
    end
    settransform!(vis["robot"], Translation(position(xbar)) ∘ LinearMap(UnitQuaternion(orientation(xbar))))

    # Compute more stuff
    # Linearize dynamics about hover
    A_cache = ForwardDiff.jacobian(x -> quad_dynamics_rk4(x, u_lin, h), x_lin)
    B_cache = ForwardDiff.jacobian(u -> quad_dynamics_rk4(x_lin, u, h), u_lin)
    f_cache = quad_dynamics_rk4(x_lin, u_lin, h)
    Ã_cache = Array(E(quat)' * A_cache * E(quat))
    B̃_cache = Array(E(quat)' * B_cache)
    f̃_cache = Array(E(quat)' * f_cache)

    # Add cache to new caches
    push!(new_caches, compute_cache!(x_lin, 1e1, Ã_cache, B̃_cache, f̃_cache, Q, R; verbose=true))
    
    # sleep(0.1)
end

solver.caches = deepcopy(new_caches)

"ihlqr converged in 550 iterations"

"ihlqr converged in 550 iterations"

"ihlqr converged in 550 iterations"

"ihlqr converged in 550 iterations"

"ihlqr converged in 550 iterations"

...

"ihlqr converged in 550 iterations"

"ihlqr converged in 550 iterations"

"ihlqr converged in 550 iterations"

"ihlqr converged in 550 iterations"

16-element Vector{Any}:
 TinyCache(10.0, [-0.5578823839562904 0.49947409133902587 … -0.13600986098137036 -1.1144986141153508; 0.5254473167737371 0.38475082316502157 … 0.13050702634646846 1.1148675876852103; 0.3523747836591953 -0.4173260084110342 … -0.0012306376087847532 -1.1154241840268992; -0.31993971647664277 -0.46689890609301943 … 0.0067334722436868475 1.115055210457044], [519989.2702408192 -117.40790936200946 … 370.78807810792915 639.0560375346466; -117.40790936201006 519880.70054136653 … -21.65907058994005 -255.4575431194231; … ; 370.78807810792983 -21.659070589940015 … 52.895096443111896 385.42924186902826; 639.0560375346479 -255.45754311942298 … 385.42924186902826 6300.278690517662], [0.0009806415865479228 9.979392937243925e-5 0.000894807053686412 0.00010020417886280017; 9.979392937243915e-5 0.0009735809033045986 0.00010146119307123287 0.0009006107227212994; 0.000894807053686412 0.00010146119307123302 0.0009762952764353637 0.00010288322527656538; 0.00010020417886280012 0.0009006

In [7]:
x_arrow_material = LineBasicMaterial(color=Colors.RGBA(0.55,0.1,0.1,1.0))
x_shaft = Cylinder(zero(Point{3, Float64}), Point(0.2, 0.0, 0.0), 0.01)
setobject!(vis["coordinate_frame_sol"][:xarrow][:shaft], x_shaft, x_arrow_material)
x_head = Cone(Point(0.2, 0.0, 0.0), Point(0.25, 0.0, 0.0), 0.025)
setobject!(vis["coordinate_frame_sol"][:xarrow][:head], x_head, x_arrow_material)

y_arrow_material = LineBasicMaterial(color=Colors.RGBA(0.1,0.55,0.1,1.0))
y_shaft = Cylinder(zero(Point{3, Float64}), Point(0.0, 0.2, 0.0), 0.01)
setobject!(vis["coordinate_frame_sol"][:yarrow][:shaft], y_shaft, y_arrow_material)
y_head = Cone(Point(0.0, 0.2, 0.0), Point(0.0, 0.25, 0.0), 0.025)
setobject!(vis["coordinate_frame_sol"][:yarrow][:head], y_head, y_arrow_material)

z_arrow_material = LineBasicMaterial(color=Colors.RGBA(0.1,0.1,0.55,1.0))
z_shaft = Cylinder(zero(Point{3, Float64}), Point(0.0, 0.0, 0.2), 0.01)
setobject!(vis["coordinate_frame_sol"][:zarrow][:shaft], z_shaft, z_arrow_material)
z_head = Cone(Point(0.0, 0.0, 0.2), Point(0.0, 0.0, 0.25), 0.025)
setobject!(vis["coordinate_frame_sol"][:zarrow][:head], z_head, z_arrow_material)

for ref in Xref
    cache_index = argmin([q_distance(c_.xlin[4:7], ref[4:7]) for c_ in solver.caches])
    settransform!(vis["coordinate_frame_sol"], Translation(ref[1:3] + [0, -0.5, 0]) ∘ LinearMap(UnitQuaternion(solver.caches[cache_index].xlin[4:7])))
    settransform!(vis["coordinate_frame_ref"], Translation(ref[1:3]) ∘ LinearMap(UnitQuaternion(ref[4:7])))
    sleep(.04)
end

In [8]:
include("../LibTinyMPC.jl")
x0 = X̃ref[:,1] + randn(nx)*0.0
# x0 = zero(X̃ref[:,1])
Xhist = zeros(nx+1, NTOTAL)
Xhist[:,1] .= [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
Uhist = zeros(nu, NTOTAL-1)

reset_solver!(solver)

for i = 1:NTOTAL-1
    # Update initial state
    solver.workspace.x[:,1] = [Xhist[:,i][1:3]; qtorp(Xhist[:,i][4:7]); Xhist[:,i][8:end]]

    # Update reference
    if i < NTOTAL - Nh
        solver.workspace.Xref = X̃ref[:,i:i+Nh-1]
    else
        solver.workspace.Xref = [X̃ref[:,i:end] repeat(X̃ref[:,end], 1, i+Nh-NTOTAL)]
    end

    # Solve
    solve_admm!(solver)
    Uhist[:,i] .= clamp.(solver.workspace.u[:,1] + u_lin_1, -1, 1)

    # Simulate with control
    Xhist[:,i+1] = quad_dynamics_rk4(Xhist[:,i], Uhist[:,i], h)








    # Visualization stuff

    # Update quadrotor transform
    xbar = RBState(quad_model, Xhist[:,i])
    if quad_model.ned
        rq = position(xbar)
        vq = linear_velocity(xbar)
        rq = SA[rq[1],-rq[2],-rq[3]]
        vq = SA[vq[1],-vq[2],-vq[3]]
        xbar = RBState(rq, RotX(pi)*orientation(xbar), vq, angular_velocity(xbar)) 
    end
    settransform!(vis["robot"], Translation(position(xbar)) ∘ LinearMap(UnitQuaternion(orientation(xbar))))
    
    # Visualize solution
    delete!(vis["xLinearized"])
    Xsol = [solver.workspace.x[:,j] for j in 1:size(solver.workspace.x, 2)]
    x_linearized = [Point(x_[1], x_[2], x_[3]+0.001) for x_ in Xsol]
    setobject!(vis["xLinearized"], Object(PointCloud(x_linearized), 
    LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))

    # Visualize quaternion at end of solution via coordinate frame
    settransform!(vis["coordinate_frame_sol"], Translation(Xsol[end][1:3]) ∘ LinearMap(UnitQuaternion(rptoq(Xsol[end][4:6]))))
    # Visualize quaternion at corresponding reference trajectory knot point via coordinate frame
    settransform!(vis["coordinate_frame_ref"], Translation(Xref[i][1:3]) ∘ LinearMap(UnitQuaternion(Xref[i][4:7])))

    # Visualize history
    xhist = [Xhist[:,j] for j in 1:i]
    delete!(vis["xHistLine"])
    xHistLine = [Point(x_[1], x_[2], x_[3]+0.001) for x_ in xhist]
    setobject!(vis["xHistLine"], Object(PointCloud(xHistLine), 
    LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))

    sleep(0.01)
end

In [28]:
delete!(vis["xHistLine"])

MeshCat Visualizer with path /meshcat/xHistLine at http://127.0.0.1:8700

In [42]:
for i in 1:size(Xhist, 2)
    settransform!(vis[coordinate_frame_name], Translation(Xhist[:,i][1:3]) ∘ LinearMap(UnitQuaternion(Xhist[:,i][4:7])))
    sleep(0.01)
end

In [68]:
using JuMP
import Ipopt
import Random
import Statistics
import Test

In [69]:
function example_rosenbrock()
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, x)
    @variable(model, y)
    @NLobjective(model, Min, (1 - x)^2 + 100 * (y - x^2)^2)
    optimize!(model)
    Test.@test objective_value(model) ≈ 0.0 atol = 1e-10
    Test.@test value(x) ≈ 1.0
    Test.@test value(y) ≈ 1.0
    return
end
example_rosenbrock()

In [247]:
let
    include("../quaternion-stuff.jl")
    function quad_dynamics(x, u)
        r = x[1:3]
        q = x[4:7] #/ norm(x[4:7])
        v = x[8:10]
        ω = x[11:13]
        Q = qtoQ(q)
    
        ṙ = v
        q̇ = 0.5 * L(q) * H * ω

        v̇ = [0; 0; -g] + (1 / m) * Q * [zeros(2, 4); kt * ones(1, 4)] * u
        # v̇ = zeros(3)
    
        # ω̇ = J \ (-hat(ω) * J * ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km] * u)
        ω̇ = zeros(3)
    
        return [ṙ; q̇; v̇; ω̇ ]
    end
    
    # function quad_dynamics_rk4(x, u, h)
    #     #RK4 integration with zero-order hold on u
    #     f1 = quad_dynamics(x, u)
    #     f2 = quad_dynamics(x + 0.5 * h * f1, u)
    #     f3 = quad_dynamics(x + 0.5 * h * f2, u)
    #     f4 = quad_dynamics(x + h * f3, u)
    #     xn = x + (h / 6.0) * (f1 + 2 * f2 + 2 * f3 + f4)
    #     # xn[4:7] .= xn[4:7] / norm(xn[4:7]) #re-normalize quaternion
    #     return xn
    # end

    function objective_function(x, u)
        J = 0
        for i = 1:size(x, 2)
            quat = qtorp(L(Xref[i][4:7])' * x[4:7,i])
            x_diff = [x[1:3,i] - Xref[i][1:3]; quat; x[8:end,i] - Xref[i][8:end]]
            J += x_diff' * Q * x_diff
        end

        for i = 1:size(u, 2)
            # J += (u[i,end] - u_ref[i,end])' * R * (u[i,end] - u_ref[i,end])
            J += u[:,i]'*u[:,i]
        end
        return J
    end
    
    nx = 13
    nu = 4
    traj_length = length(Xref)
    h = 1/100
    

    model = Model(Ipopt.Optimizer)
    register(model, :objective_function, 2, objective_function; autodiff = true)
    register(model, :quad_dynamics, 2, quad_dynamics; autodiff = true)
    set_silent(model)
    @variable(model, x[1:nx, 1:traj_length])
    @variable(model, u[1:nu, 1:traj_length-1])
    for i in 1:nx # initial and final conditions
        fix(x[i,1], Xref[1][i])
        fix(x[i,end], Xref[end][i])
    end
    function obj_x(x)
        J = 0
        for i = 1:size(x, 2)
            quat = qtorp(L(Xref[i][4:7])' * x[4:7,i])
            x_diff = [x[1:3,i] - Xref[i][1:3]; quat; x[8:end,i] - Xref[i][8:end]]
            J += x_diff[:,i]'*x_diff[:,i]
        end
        return J
    end
    @NLobjective(model, Min, obj_x(x))
    # @NLobjective(model, Min, objective_function(x, u))

    # for j in 1:nx
    #     @constraint(model, [i=1:Nh-1], quad_dynamics(x[:,i], u[:,i])[j] == x[j,i+1])
    # end
end

┌ Warning: Function obj_x automatically registered with 1 arguments.
│ 
│ Calling the function with a different number of arguments will result in an
│ error.
│ 
│ While you can safely ignore this warning, we recommend that you manually
│ register the function as follows:
│ ```Julia
│ model = Model()
│ register(model, :obj_x, 1, obj_x; autodiff = true)
│ ```
└ @ JuMP /home/sam/.julia/packages/JuMP/klrjG/src/parse_nlp.jl:21


ErrorException: Unexpected array VariableRef[x[1,1] x[1,2] x[1,3] x[1,4] x[1,5] x[1,6] x[1,7] x[1,8] x[1,9] x[1,10] x[1,11] x[1,12] x[1,13] x[1,14] x[1,15] x[1,16] x[1,17] x[1,18] x[1,19] x[1,20] x[1,21] x[1,22] x[1,23] x[1,24] x[1,25] x[1,26] x[1,27] x[1,28] x[1,29] x[1,30] x[1,31] x[1,32] x[1,33] x[1,34] x[1,35] x[1,36] x[1,37] x[1,38] x[1,39] x[1,40] x[1,41] x[1,42] x[1,43] x[1,44] x[1,45] x[1,46] x[1,47] x[1,48] x[1,49] x[1,50] x[1,51] x[1,52] x[1,53] x[1,54] x[1,55] x[1,56] x[1,57] x[1,58] x[1,59] x[1,60] x[1,61] x[1,62] x[1,63] x[1,64] x[1,65] x[1,66] x[1,67] x[1,68] x[1,69] x[1,70] x[1,71] x[1,72] x[1,73] x[1,74] x[1,75] x[1,76] x[1,77] x[1,78] x[1,79] x[1,80] x[1,81] x[1,82] x[1,83] x[1,84] x[1,85] x[1,86] x[1,87] x[1,88] x[1,89] x[1,90] x[1,91] x[1,92] x[1,93] x[1,94] x[1,95] x[1,96] x[1,97] x[1,98] x[1,99] x[1,100] x[1,101] x[1,102] x[1,103] x[1,104] x[1,105] x[1,106] x[1,107] x[1,108] x[1,109] x[1,110] x[1,111] x[1,112] x[1,113] x[1,114] x[1,115] x[1,116] x[1,117] x[1,118] x